<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainCountries01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

# df_annotations = pd.read_csv("all_annotations.csv")


# Get data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open("/content/drive/MyDrive/data/protocols.pkl.gz", "rb") as f:
    file_to_pages = pkl.load(f)

In [4]:
import spacy
nlp = spacy.blank("en")

In [5]:
df_annotations= pd.read_csv("/content/drive/MyDrive/data/filtered_for_phase_arms_subjects_02.csv.bz2")

In [6]:
!pip install country_named_entity_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 4.6 MB/s 
  Created wheel for pycountry: filename=pycountry-22.1.10-py2.py3-none-any.whl size=10595786 sha256=17154abd8f467b836abaede7f400e04ffa63fafb989d0c36816922b202cdc5ce
  Stored in directory: /root/.cache/pip/wheels/f7/8f/9c/b070d7376caf2beb0685bf72578106b2fd57019ed57d84f126
Successfully built pycountry


In [7]:
from country_named_entity_recognition import find_countries

In [8]:
import re

import pycountry

demonym_to_country_code = {'ARUBAN': 'AW',
                           'AFGHAN': 'AF',
                           'ANGOLAN': 'AO',
                           'ANGUILLIAN': 'AI',
                           'ALBANIAN': 'AL',
                           'EMIRATI': 'AE',
                           'ARGENTINEAN': 'AR',
                           'ARMENIAN': 'AM',
                           'AMERICAN SAMOAN': 'AS',
                           'FRENCH': 'PM',
                           'ANTIGUAN': 'AG',
                           'BARBUDAN': 'AG',
                           'AUSTRALIAN': 'AU',
                           'AUSTRIAN': 'AT',
                           'AZERBAIJANI': 'AZ',
                           'BURUNDIAN': 'BI',
                           'BELGIAN': 'BE',
                           'BENINESE': 'BJ',
                           'BURKINABE': 'BF',
                           'BANGLADESHI': 'BD',
                           'BULGARIAN': 'BG',
                           'BAHRAINI': 'BH',
                           'BAHAMIAN': 'BS',
                           'BOSNIAN': 'BA',
                           'HERZEGOVINIAN': 'BA',
                           'BELARUSIAN': 'BY',
                           'BELIZEAN': 'BZ',
                           'BERMUDIAN': 'BM',
                           'BOLIVIAN': 'BO',
                           'BRAZILIAN': 'BR',
                           'BARBADIAN': 'BB',
                           'BRUNEIAN': 'BN',
                           'BHUTANESE': 'BT',
                           'MOTSWANA': 'BW',
                           'CENTRAL AFRICAN': 'CF',
                           'CANADIAN': 'CA',
                           'COCOS ISLANDER': 'CC',
                           'SWISS': 'CH',
                           'CHILEAN': 'CL',
                           'CHINESE': 'MO',
                           'IVORIAN': 'CI',
                           'CAMEROONIAN': 'CM',
                           'CONGOLESE': 'CG',
                           'COOK ISLANDER': 'CK',
                           'COLOMBIAN': 'CO',
                           'COMORAN': 'KM',
                           'CAPE VERDIAN': 'CV',
                           'COSTA RICAN': 'CR',
                           'CUBAN': 'CU',
                           'CHRISTMAS ISLAND': 'CX',
                           'CAYMANIAN': 'KY',
                           'CYPRIOT': 'CY',
                           'CZECH': 'CZ',
                           'GERMAN': 'DE',
                           'DJIBOUTI': 'DJ',
                           'DOMINICAN': 'DO',
                           'DANISH': 'DK',
                           'ALGERIAN': 'DZ',
                           'ECUADOREAN': 'EC',
                           'EGYPTIAN': 'EG',
                           'ERITREAN': 'ER',
                           'SAHRAWI': 'EH',
                           'SPANISH': 'ES',
                           'ESTONIAN': 'EE',
                           'ETHIOPIAN': 'ET',
                           'FINNISH': 'FI',
                           'FIJIAN': 'FJ',
                           'FALKLAND ISLANDER': 'FK',
                           'FAROESE': 'FO',
                           'MICRONESIAN': 'FM',
                           'GABONESE': 'GA',
                           'BRITISH': 'GB',
                           'GEORGIAN': 'GE',
                           'CHANNEL ISLANDER': 'JE',
                           'GHANAIAN': 'GH',
                           'GIBRALTAR': 'GI',
                           'GUINEAN': 'GN',
                           'GUADELOUPIAN': 'GP',
                           'GAMBIAN': 'GM',
                           'GUINEA-BISSAUAN': 'GW',
                           'EQUATORIAL GUINEAN': 'GQ',
                           'GREEK': 'GR',
                           'GRENADIAN': 'GD',
                           'GREENLANDIC': 'GL',
                           'GUATEMALAN': 'GT',
                           'GUAMANIAN': 'GU',
                           'GUYANESE': 'GY',
                           'HEARD AND MCDONALD ISLANDER': 'HM',
                           'HONDURAN': 'HN',
                           'CROATIAN': 'HR',
                           'HAITIAN': 'HT',
                           'HUNGARIAN': 'HU',
                           'INDONESIAN': 'ID',
                           'MANX': 'IM',
                           'INDIAN': 'IO',
                           'IRISH': 'IE',
                           'IRANIAN': 'IR',
                           'IRAQI': 'IQ',
                           'ICELANDER': 'IS',
                           'ISRAELI': 'IL',
                           'ITALIAN': 'IT',
                           'JAMAICAN': 'JM',
                           'JORDANIAN': 'JO',
                           'JAPANESE': 'JP',
                           'KAZAKHSTANI': 'KZ',
                           'KENYAN': 'KE',
                           'KIRGHIZ': 'KG',
                           'CAMBODIAN': 'KH',
                           'I-KIRIBATI': 'KI',
                           'KIRIBATI': 'KI',
                           'KITTIAN AND NEVISIAN': 'KN',
                           'SOUTH KOREAN': 'KR',
                           'KUWAITI': 'KW',
                           'LAOTIAN': 'LA',
                           'LEBANESE': 'LB',
                           'LIBERIAN': 'LR',
                           'LIBYAN': 'LY',
                           'SAINT LUCIAN': 'LC',
                           'LIECHTENSTEINER': 'LI',
                           'SRI LANKAN': 'LK',
                           'MOSOTHO': 'LS',
                           'LITHUANIAN': 'LT',
                           'LUXEMBOURGER': 'LU',
                           'LATVIAN': 'LV',
                           'MOROCCAN': 'MA',
                           'MONEGASQUE': 'MC',
                           'MOLDOVAN': 'MD',
                           'MALAGASY': 'MG',
                           'MALDIVAN': 'MV',
                           'MEXICAN': 'MX',
                           'MARSHALLESE': 'MH',
                           'MACEDONIAN': 'MK',
                           'MALIAN': 'ML',
                           'MALTESE': 'MT',
                           'MONGOLIAN': 'MN',
                           'AMERICAN': 'US',
                           'MOZAMBICAN': 'MZ',
                           'MAURITANIAN': 'MR',
                           'MONTSERRATIAN': 'MS',
                           'MAURITIAN': 'MU',
                           'MALAWIAN': 'MW',
                           'MALAYSIAN': 'MY',
                           'NAMIBIAN': 'NA',
                           'NEW CALEDONIAN': 'NC',
                           'NIGERIEN': 'NE',
                           'NORFOLK ISLANDER': 'NF',
                           'NIGERIAN': 'NG',
                           'NICARAGUAN': 'NI',
                           'NIUEAN': 'NU',
                           'DUTCH': 'NL',
                           'NORWEGIAN': 'SJ',
                           'NEPALESE': 'NP',
                           'NAURUAN': 'NR',
                           'NEW ZEALANDER': 'NZ',
                           'OMANI': 'OM',
                           'PAKISTANI': 'PK',
                           'PANAMANIAN': 'PA',
                           'PITCAIRN ISLANDER': 'PN',
                           'PERUVIAN': 'PE',
                           'FILIPINO': 'PH',
                           'PALAUAN': 'PW',
                           'PAPUA NEW GUINEAN': 'PG',
                           'POLISH': 'PL',
                           'PUERTO RICAN': 'PR',
                           'NORTH KOREAN': 'KP',
                           'PORTUGUESE': 'PT',
                           'PARAGUAYAN': 'PY',
                           'FRENCH POLYNESIAN': 'PF',
                           'QATARI': 'QA',
                           'ROMANIAN': 'RO',
                           'RUSSIAN': 'RU',
                           'RWANDAN': 'RW',
                           'SAUDI ARABIAN': 'SA',
                           'SUDANESE': 'SD',
                           'SENEGALESE': 'SN',
                           'SINGAPOREAN': 'SG',
                           'SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDER': 'GS',
                           'SAINT HELENIAN': 'SH',
                           'SOLOMON ISLANDER': 'SB',
                           'SIERRA LEONEAN': 'SL',
                           'SALVADORAN': 'SV',
                           'SAMMARINESE': 'SM',
                           'SOMALI': 'SO',
                           'SOUTH SUDANESE': 'SS',
                           'SAO TOMEAN': 'ST',
                           'SURINAMER': 'SR',
                           'SLOVAK': 'SK',
                           'SLOVENE': 'SI',
                           'SWEDISH': 'SE',
                           'SWAZI': 'SZ',
                           'SEYCHELLOIS': 'SC',
                           'SYRIAN': 'SY',
                           'CHADIAN': 'TD',
                           'TOGOLESE': 'TG',
                           'THAI': 'TH',
                           'TADZHIK': 'TJ',
                           'TAJIK': 'TJ',
                           'TOKELAUAN': 'TK',
                           'TURKMEN': 'TM',
                           'EAST TIMORESE': 'TL',
                           'TONGAN': 'TO',
                           'TRINIDADIAN': 'TT',
                           'TUNISIAN': 'TN',
                           'TURKISH': 'TR',
                           'TUVALUAN': 'TV',
                           'TAIWANESE': 'TW',
                           'TANZANIAN': 'TZ',
                           'UGANDAN': 'UG',
                           'UKRAINIAN': 'UA',
                           'URUGUAYAN': 'UY',
                           'UZBEKISTANI': 'UZ',
                           'SAINT VINCENTIAN': 'VC',
                           'VENEZUELAN': 'VE',
                           'VIETNAMESE': 'VN',
                           'NI-VANUATU': 'VU',
                           'WALLIS AND FUTUNA ISLANDER': 'WF',
                           'SAMOAN': 'WS',
                           'YEMENI': 'YE',
                           'SOUTH AFRICAN': 'ZA',
                           'ZAMBIAN': 'ZM',
                           'ZIMBABWEAN': 'ZW'}

demonym_to_country = dict(
    [demonym, pycountry.countries.lookup(alpha_2)]
    for demonym, alpha_2 in demonym_to_country_code.items()
)

people = {"adults", "men", "women", "children", "infants", "toddlers", "sex workers",
          "communities", "community",
          "population",
          "males", "male adults", "females",
          "female adults", "individuals",
          "volunteers", "babies", "teenagers", "adolescents", "pregnant"}

country_pattern = "(" + "|".join(demonym_to_country_code) + ")"
people_pattern = "(" + "|".join(people) + ")"

demonym_regex = re.compile(f"(?i)\\b(?:{country_pattern}(?:s|\\s*{people_pattern}))\\b")


def find_demonyms(text: str) -> list:
    """
    Find the nationalities mentioned in the text that clearly refer to study subjects, e.g. Samoan males.
    :param text: the text of the protocol page
    :return: nationalities found as a list of tuples (PyCountry object and Match object).
    """
    country_matches = []

    matches = demonym_regex.finditer(text)
    for match in matches:
        matched_country = demonym_to_country[match.groups()[0].upper()]
        country_matches.append((matched_country, match))

    return country_matches


In [43]:
def get_contexts(pages):

    country_to_contexts = {}

    for page_no, page_text in enumerate(pages):
        countries = find_countries(page_text)

        demonyms = find_demonyms(page_text)

        matches = []
        for country, match in countries:
            matches.append((country, match, "USA"))
        for country, match in demonyms:
            first_word = match.group().split(" ")[0]
            if first_word.endswith("s"):
              substitute = re.sub(first_word, "Americans", match.group())
            else:
              substitute = re.sub(first_word, "American", match.group())
            matches.append((country, match, substitute))

        for country, match, substitute in matches:
            if country.alpha_2 not in country_to_contexts:
                country_to_contexts[country.alpha_2] = []

            start = match.start() - 100
            end = match.end() + 100
            if start < 0:
                start = 0
            if end > len(page_text) - 1:
                end = len(page_text)

            left_context = page_text[start:match.start()]
            left_context = re.sub(r'the\s*$', '', left_context)
            context = left_context + substitute + page_text[match.end():end]
            context = re.sub(r'^\w+|\w+$', '', context).strip()
            country_to_contexts[country.alpha_2].append(context)
    
    return country_to_contexts

In [ ]:
all_contexts = []
for idx in range(len(df_annotations)):
  if not pd.isna(df_annotations.file.iloc[idx]):
    pages = file_to_pages[df_annotations.file.iloc[idx]]
    contexts = get_contexts(pages)
  else:
    contexts = {}
  all_contexts.append(contexts)

In [45]:
train_examples = []
test_examples = []
train_labels = []
test_labels = []
for idx in range(len(df_annotations)):
  gt = set(df_annotations.country.iloc[idx].split(","))
  is_found = False
  for country_code, contexts in all_contexts[idx]:
     if country_code in gt:
       is_found = True
  if is_found:
    if df_annotations.train_val.iloc[idx] == "train":
      e = train_examples
      l = train_labels
    else:
      e = test_examples
      l = test_labels
    for country_code, contexts in all_contexts[idx].items():
      text= " ".join(contexts)
      e.append(text)
      l.append(int(country_code in gt))    

{'US': ['sickle cell anemia (SCA) is a\ndevastating medical disorder.  Over 50,000 affected persons in USA have inherited\nSCA, and the vast majority of them are African-American.  To date, there have been',
  'the proposed answers must be sensitive to the history of medical investigation\ninvolving African-Americans.  \n\nHighest priority must be given to patient protection and to assuring that patients’ families',
  'predominance of sickle hemoglobin (HbS).  SCD is one of the most common\n\ninherited diseases in USA, affecting approximately 1 in 375 African-American live\n\nbirths.  Currently it is estimated that',
  '-American live\n\nbirths.  Currently it is estimated that there are over 75,000 persons in USA with SCD.\n\nA single inherited amino acid substitution in beta globin results in the formation of',
  'with HU (mean 5.6 years) and had no cases of secondary malignancy (Triadou et al,\n\n1994).  In USA, some adults with SCA have received HU therapy for over 10 years; no\n\nc

In [45]:
train_examples = np.asarray(train_examples)
test_examples = np.asarray(test_examples)
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[tf.metrics.CategoricalAccuracy(name='accuracy')],
              run_eagerly=True)

In [ ]:
partial_x_train = train_examples
x_val = test_examples

partial_y_train = train_labels
y_val = test_labels

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

In [ ]:
with open("/content/drive/MyDrive/data/countries_contexts.pkl.gz", "wb") as f:
    pkl.dump(all_contexts, f)

In [ ]:
import operator


value_to_scores = []

for idx in range(len(df_annotations)):
  if df_annotations.train_val.iloc[idx] == "val":
    contexts = all_contexts[idx]
    candidate_values = list(contexts.keys())
    input_texts = np.asarray(list([" ".join(c) for c in contexts.values()  ] ))
    scores = model.predict(input_texts)

    value_to_score = {}
    for j in range(len(contexts)):
      value_to_score[candidate_values[j]] = scores[j][0]
    value_to_scores.append(value_to_score)

In [ ]:
pred_proba = []

for idx in range(len(df_annotations[df_annotations.train_val == "val"])):
  value_to_score = value_to_scores[idx]
  winning_values = sorted(value_to_score.items(), key=operator.itemgetter(1), reverse=True)
  if winning_values[0][1] < 0.5:
    winning_values = winning_values[:1]
  else:
    winning_values = [v for v in winning_values if v[1] > 0.5]
  pred_proba.append(",".join(sorted(set(winning_values))))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
df_val = df_annotations[df_annotations.train_val == "val"]

In [ ]:
df_val["y_pred"] = pred_proba

In [ ]:
accuracy_score(df_val.country, df_val.y_pred)